# Containers

### What are the goals of containers?

- Isolate full computing environments (not only Python packages)
- Provide a mechanism to encapsulate environments in a self-contained unit that can run anywhere

### Why do we need containers?
- Science Reproducibility 
- Collaboration with your colleagues
- Installing new software without fear of breaking you environment.

### Difference between Virtual Machines and Containers
The main idea is the same &ndash; isolate the computing environment, but they differ.

- Virtual Machine:
    - emulate whole computer system (software+hardware)
    - run on top of a physical machine using a hypervisor
    - hypervisor shares and manages hardware of the host and executes the guest operating system
    - guest machines are completely isolated and have dedicated resources

<img src="assets/VM.png" alt="metadata" width='300px'>

- Containers (Docker)
    - share the host system’s kernel with other containers
    - each container gets its own isolated user space
    - only bins and libs are created from scratch
    - **containers are very lightweight and fast to start up**
    
<img src="assets/Container.png" alt="metadata" width='300px'>

You can find more on:
 - [Docker website](https://www.docker.com/what-container)
 - [Beginner friendly intro to VMs and Docker](https://medium.freecodecamp.org/a-beginner-friendly-introduction-to-containers-vms-and-docker-79a9e3e119b)


# [Docker](docker.com)
- leading software container platform
- an open-source project
- it runs now on Mac OS X and Windows (Pro versions) without VM

### Using Docker
- you can use existing Docker images, many public images can be found on [Docker Hub](https://hub.docker.com/)
    - example of running existing images can be found [here](http://nipy.org/workshops/2017-03-boston/lectures/lesson-container/#29)
- you can create your own images by writing `Dockerfile`
    - the simplest examples you can find [here](http://nipy.org/workshops/2017-03-boston/lectures/lesson-container/#31)
    - [Dockerfile official reference](https://docs.docker.com/engine/reference/builder/#from)
    - there are also tools that helps you generate Dockerfile, e.g. [Neurodocker](https://github.com/kaczmarj/neurodocker)

### Docker and Singularity
- Docker:
    - docker can escalate privileges, so you can be effectively treated as a root on the host system
    - this is usually not supported by administrators from HPC centers
- [Singularity](http://singularity.lbl.gov/):
    - a container solution created for scientific and application driven workloads
    - supports existing and traditional HPC resources
    - a user inside a Singularity container is the same user as outside the container
    - but you can use Vagrant to create a container (you have root privileges on your VM!)
    - can run (and modify!) existing Docker containers
    - running VM is required on OSX and Windows

## Docker for the Reproducible Python

If you want to try using Docker, we created a Docker image that could be used for this tutorial. Below you can see the Neurodocker command that was used to create a Dockerfile.                                                                                                    
```
docker run --rm kaczmarj/neurodocker:master generate docker \
  --base neurodebian:stretch-non-free \
  --pkg-manager apt \
  --install git git-annex-standalone git-annex-remote-rclone vim emacs-nox nano \
  --user=repropython \
  --miniconda \
    env_name="repropython" \
    conda_install="python=3.6 pytest jupyter jupyterlab cookiecutter numpy                                         
                   pandas matplotlib" \
    pip_install="nbval nbdime recipy jupyter-repo2docker datalad" \
    activate=true \
  --run 'mkdir -p ~/.jupyter && echo c.NotebookApp.ip = \"0.0.0.0\" > ~/.jupyter/jupyter_notebook_config.py' \
  --user=root \
  --run 'mkdir /ReproduciblePython && chmod 777 /ReproduciblePython && chmod a+s /ReproduciblePython' \
  --run 'rm -rf /opt/miniconda-latest/pkgs/*' \
  --user=repropython \
  --workdir /home/repropython \
  --cmd jupyter-lab > Dockerfile
 ```


You can create Dockerfile and Docker image using the command, but you can also download the ready to use image from DockerHub by typing in your terminal:
```
docker pull djarecka/repropython
```

In order to run the container you can type in your terminal:
```
docker run -it --rm -p 8888:8888 djarecka/repropython
```
- `-it` flag tells docker that it should open an interactive container instance.
- `--rm` flag tells docker that the container should automatically be removed after we close docker.
- `-p` flag specifies which port we want to make available for docker.

`jupyter-lab` is set as the default command, so this command will start `jupyter-lab` automatically. The URL (that looks like `http://localhost:8888/?token=0312c1ef3b61d7a44ff5346d3d150c23249a548850e13868`) should be paste to your browser.

You can also open the container with `bash` by typing:
```
docker run -it --rm -p 8888:8888 djarecka/repropython bash
```


### Opening the notebooks from the tutorial

You might notice that after you open `jupyter-lab` you don't have any notebooks, because the content of the tutorial was not copied to the container. However you can always mount the directory with your current local version of notebooks by adding `-v` flag:
```
docker run -it --rm -v /path/to/your/local/directory/ReproduciblePython:/home/repropython/ReproduciblePython  -p 8888:8888 djarecka/repropython
```

In [1]:
from IPython.core.display import HTML


def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

## Creating Docker containers using repo2docker

The package [repo2docker](http://repo2docker.readthedocs.io/en/latest/usage.html#running-repo2docker-locally) allows you to generate a Docker container from a repository (e.g. GitHub). 

To get started you need to make sure you have any of the following files:
- Dockerfile
- environment.yml
- requirements.yml
- REQUIRE (Julia packages, plus it needs an environment.yml as well)
- postBuild


If you followed along the course you should now have a conda environment file `environment.yml`:

```yaml
name: testenv
channels:
  - conda-forge
  - defaults
dependencies:
  - python>3.6
  - pytest
  - pandas
  - matplotlib
  - pip:
    - nbval
```


### Getting Dockerized

<div class = 'info'> You need to have Docker working in your computer before proceeding </div>

`repo2docker` follows the next two steps:
1. Builds a Docker image from a git repo
2. Runs a Jupyter server within the image (so you can explore the repository's content)

From your command line type:
```bash
$ jupyter-repo2docker <url to the repository>
```

The process might take a while, and once completed you should see message like this in your terminal
```bash
Copy/paste this URL into your browser when you connect for the first time,
to login with a token:
    http://0.0.0.0:36511/?token=f94f8fabb92e22f5bfab116c382b4707fc2cade56ad1ace0
```

In [1]:
from IPython.core.display import HTML


def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()